# Random Forest

Even more effective for predictions than decision trees, random forests are useful and powerful ensemble methods for classifying and regressing data. Essentially, a random set of the features are taken to build decision trees. Trees are also built using bagged data which are samples of the data taken with replacement. Many trees are grown (sometimes their maximum depth is also specified) and then all trees vote on the response. Each tree's vote generally has the same weight.

Though most of the the time, Sci-Kit Learn can be used to implement this machine learning method simply, it is useful to take a look at the inner workings of these algorithms. This notebook will first introduce a decision tree from scratch. It will then show an example using SK Learn.

## Random Forest From Scratch Using HMEQ Data

Firstly, we will go through a random forest made from scratch. The following code is based on the material originally created by Sebastian Mantey for the Iris Dataset whose GitHub is here: https://github.com/SebastianMantey/Random-Forest-from-Scratch. You can also search his video on YouTube. The code has been adapted to this course's format and fit to HMEQ Dataset.

### Load Relevant Libraries & Data

The first decision tree will be coded from scratch using only pandas and NumPy. There are faster ways to do this using Sci-Kit Learn which is a l

In [1]:
import numpy as np
import pandas as pd
from pprint import pprint

import random #needed for bagging

In [2]:
filename = 'https://github.com/Humboldt-WI/bads/blob/master/data/hmeq_modeling.csv?raw=true'
df = pd.read_csv(filename, header = 0, index_col = 0)

In [3]:
df.head()

,BAD,LOAN,MORTDUE,VALUE,YOJ,CLAGE,NINQ,CLNO,DEBTINC,DEROGzero,REASON_HomeImp,REASON_IsMissing,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self,DELINQcat_1,DELINQcat_1+
index,,,,,,,,,,,,,,,,,,,
0,True,-1.832283,-1.295882,-1.335526,0.266788,-1.075278,-0.065054,-1.297476,0.137456,True,1,0,0,1,0,0,0,0,0
1,True,-1.810666,-0.013474,-0.672699,-0.236615,-0.723092,-0.826792,-0.756608,0.137456,True,1,0,0,1,0,0,0,0,1
2,True,-1.789048,-1.654549,-1.839275,-0.668103,-0.368769,-0.065054,-1.189302,0.137456,True,1,0,0,1,0,0,0,0,0
3,True,-1.789048,-0.159552,-0.202559,-0.236615,-0.061033,-0.065054,-0.107566,0.137456,True,0,1,0,1,0,0,0,0,0
4,False,-1.767431,0.791699,0.311107,-0.811933,-1.088528,-0.826792,-0.756608,0.137456,True,1,0,1,0,0,0,0,0,0


In [4]:
X = df.drop(['BAD'], axis=1) #code the variables in the most standard way for your usage
y = df[['BAD']]

X.head() #inspect that variables were correctly separated

,LOAN,MORTDUE,VALUE,YOJ,CLAGE,NINQ,CLNO,DEBTINC,DEROGzero,REASON_HomeImp,REASON_IsMissing,JOB_Office,JOB_Other,JOB_ProfExe,JOB_Sales,JOB_Self,DELINQcat_1,DELINQcat_1+
index,,,,,,,,,,,,,,,,,,
0,-1.832283,-1.295882,-1.335526,0.266788,-1.075278,-0.065054,-1.297476,0.137456,True,1,0,0,1,0,0,0,0,0
1,-1.810666,-0.013474,-0.672699,-0.236615,-0.723092,-0.826792,-0.756608,0.137456,True,1,0,0,1,0,0,0,0,1
2,-1.789048,-1.654549,-1.839275,-0.668103,-0.368769,-0.065054,-1.189302,0.137456,True,1,0,0,1,0,0,0,0,0
3,-1.789048,-0.159552,-0.202559,-0.236615,-0.061033,-0.065054,-0.107566,0.137456,True,0,1,0,1,0,0,0,0,0
4,-1.767431,0.791699,0.311107,-0.811933,-1.088528,-0.826792,-0.756608,0.137456,True,1,0,1,0,0,0,0,0,0


In [5]:
y.head()

,BAD
index,
0,True
1,True
2,True
3,True
4,False


In [6]:
print(type(X), type(y), X.shape, y.shape) # double check that types and dimensions are correct before proceeding

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> (5960, 18) (5960, 1)


### Required Helper Functions for Individual Trees

Again, here's how a summary of how individual trees in our forest will work:

Many potential ways to split the data are calculated (eg. we are going to go through all unique values of each column and finding the midpoint between sequential values). For each potential split, we evaluate whether the target variable has more homogeneity in each leaf. This is done by calculating 'impurity' of the parent node and comparing it with the sum of the impurity of the child nodes. There are three major impurity functions: entropy, gini and misclassification. We will be using entropy in our example. The split which yields the lowest impurity is chosen and the process is repeated for the new nodes (this is recursion). 

The method of choosing the split which yields the lowest impurity is called the greedy search method. The following functions will help the decision tree implement greedy search tactics on the data. The algorithm stops either when purity in each node is reached or when it has reached a maximum depth (max amount of recursions we allow) specified in our function.

Many functions for the tree are not found in Python packages and it is cleaner to write them out first then put them together in our main algorithm. Each function below does a specific action which will be used in our final tree function at the end.

In [7]:
def check_purity(y):
    
    'checks if a leaf node is perfectly pure, in other words, if the leaf node contains only one class'
    
    unique_classes = np.unique(y) #count number of classes in section of data

    if len(unique_classes) == 1: #check if the node is pure
        return True
    else:
        return False

In [8]:
def classify_data(y):
    
    'classifies data according to the majority class of each leaf'
    
    unique_classes, counts_unique_classes = np.unique(y, return_counts=True)
    #returns classes and no. of obs per class

    index = counts_unique_classes.argmax() #index of class with most obs
    classification = unique_classes[index] #class chosen for classification which is class with most obs
    
    return classification

In [9]:
def get_potential_splits(X, random_subspace = None):
    
    'first, takes every unique value of every feature in the feature space, then finds the midpoint between each value'
    'modified to add random_subspace for random forest'
    
    potential_splits = {}
    _, n_columns = X.shape #don't need rows, we choose the column to split on
    # only need second value of .shape which is columns
    column_indices = list(range(n_columns))
    
    if random_subspace and random_subspace <= len(column_indices): #randomly chosen features
        column_indices = random.sample(population=column_indices, k=random_subspace)
    
    for column_index in column_indices:
        potential_splits[column_index] = [] 
        values = X[:, column_index] 
        unique_values = np.unique(values) #get all unique values in each column

        for index in range(len(unique_values)): #all unique feature values
            if index != 0: #skip first value, we need the difference between next values
                current_value = unique_values[index]
                previous_value = unique_values[index - 1] #find a value and the next smallest value
                potential_split = (current_value + previous_value) / 2 #find difference between the two as a potential split
                
                #consider all values which lie between two values as a potential split
                
                potential_splits[column_index].append(potential_split)
    
    return potential_splits

In [10]:
def split_data(X, y, split_column, split_value):
    
    'splits data based on specific value, will yield both a split for the features X and target y'
    
    split_column_values = X[:, split_column]

    X_below = X[split_column_values <= split_value] #partitions data according to split values from previous functions
    X_above = X[split_column_values >  split_value]
    
    y_below = y[split_column_values <= split_value]
    y_above = y[split_column_values >  split_value]
    
    return X_below, X_above, y_below, y_above

In [11]:
def calculate_entropy(y):
    
    'calculates entropy for each partition of data'
    
    _, counts = np.unique(y, return_counts=True)

    probabilities = counts / counts.sum() #probability of each class
    entropy = sum(probabilities * -np.log2(probabilities)) #could replace with Gini impurity or misclassification
     
    return entropy

In [12]:
def calculate_overall_entropy(y_below, y_above): 
    
    'calculates the total entropy after each split'
       
    n = len(y_below) + len(y_above)
    p_data_below = len(y_below) / n
    p_data_above = len(y_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(y_below)
                      + p_data_above * calculate_entropy(y_above))
    
    return overall_entropy

In [13]:
def determine_best_split(X, y, potential_splits):
    
    'selects which split lowered entropy the most'
    
    overall_entropy = 9999 #set arbitrarily high, the function will loop over and replace this with lower impurity values
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            X_below, X_above, y_below, y_above = split_data(X, y, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(y_below, y_above)
            
            #goes through each potential split and only updates if it lowers entropy

            if current_overall_entropy <= overall_entropy: 
                overall_entropy = current_overall_entropy #updates only if lower entropy split found, in the ned this is greedy search
                best_split_column = column_index
                best_split_value = value
            
    
    return best_split_column, best_split_value

The tree will now implement the helper functions and display the decision which yielded the best split if printed.

In [14]:
def decision_tree_algorithm(X, y, counter=0, min_samples=2, max_depth=5, random_subspace = None): 
    'same function as in the Decision Tree notebook but now we add random_subspace argument'
    # data preparation
    if counter == 0: # counter tells us how deep the tree is, this is before the tree is initiated
        global COLUMN_HEADERS
        COLUMN_HEADERS = X.columns
        X = X.values #change all to NumPy array for faster calculations
        y = y.values
    else:
        data = X #if we have started the tree, X should already be a NumPy array from the code above 
    
    # base cases
    if (check_purity(y)) or (len(X) < min_samples) or (counter == max_depth):
        classification = classify_data(y)
        
        return classification
    
    # recursive part
    else:    
        counter += 1 #tells us how deep the tree is

        # helper functions 
        potential_splits = get_potential_splits(X, random_subspace) #check for all possible splits
        best_split_column, best_split_value = determine_best_split(X, y, potential_splits) #select best split based on entropy
        X_below, X_above, y_below, y_above = split_data(X, y, best_split_column, best_split_value) #execute best split
        
        # code to explain decisions made by tree to users
        feature_name = COLUMN_HEADERS[best_split_column]
        question = "{} <= {}".format(feature_name, best_split_value) #initiate explanation of split
        sub_tree = {question: []}
        
        # pull answers from tree
        yes_answer = decision_tree_algorithm(X_below, y_below, counter, min_samples, max_depth, random_subspace)
        no_answer = decision_tree_algorithm(X_above, y_above, counter, min_samples, max_depth, random_subspace)
        
        # ensure explanation actually shows useful information
        if yes_answer == no_answer: #if decisions are the same, only display one
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

### Random Forest Algorithm

In [15]:
def bootstrapping(X, y, n_bootstrap):
    bootstrap_indices = np.random.randint(low=0, high=len(X), size=n_bootstrap)
    X_bootstrapped = X.iloc[bootstrap_indices]
    y_bootstrapped = y.iloc[bootstrap_indices]
    
    return X_bootstrapped, y_bootstrapped

In [16]:
def random_forest_algorithm(X, y, n_trees, n_bootstrap, n_features, dt_max_depth):
    forest = []
    for i in range(n_trees):
        X_bootstrapped, y_bootstrapped = bootstrapping(X, y, n_bootstrap)
        tree = decision_tree_algorithm(X_bootstrapped, y_bootstrapped, max_depth=dt_max_depth, random_subspace=n_features)
        forest.append(tree)
    
    return forest

In [17]:
def predict_example(example, tree, counter=0):
    
    if counter == 0 and isinstance(tree, dict) == False: #if choosing very shallow trees, many may just always predict one way
        return tree
    
    else:
        counter += 1
    
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return predict_example(example, residual_tree)

    
# 3.2 All examples of the test data
def decision_tree_predictions(test_df, tree):
    predictions = test_df.apply(predict_example, args=(tree,), axis=1)
    return predictions

In [18]:
def random_forest_predictions(test_df, forest):
    df_predictions = {}
    for i in range(len(forest)):
        column_name = "tree_{}".format(i) #key for dictionary
        predictions = decision_tree_predictions(test_df, tree=forest[i]) #predictions from trees
        df_predictions[column_name] = predictions #insert predictions into dictionary

    df_predictions = pd.DataFrame(df_predictions) #change dictionary to pandas DF
    random_forest_predictions = df_predictions.mode(axis=1)[0] #take mode of predictions over trees for final prediction
    #if there is an even number of predictions, just default to the first value (very unlikely with many trees)
    
    return random_forest_predictions

### Grow Forest

Test the forest and display the decisions with a shallow depth.

In [19]:
forest = random_forest_algorithm(X, y, n_trees=50, n_bootstrap=800, n_features=8, dt_max_depth=5)

In [20]:
forest

[{'DEBTINC <= 0.08903312622836487': [{'LOAN <= -1.2864426460968665': [{'JOB_Other <= 0.5': [False,
       True]},
     {'NINQ <= 2.6010281608341117': [{'CLNO <= -1.5679101636166737': [{'YOJ <= -1.1715060559617136': [False,
           True]},
         False]},
       {'YOJ <= -0.3804444188483026': [{'MORTDUE <= -0.27440674911385166': [False,
           True]},
         False]}]}]},
   {'DEBTINC <= 0.13832815324590006': [{'DELINQcat_1+ <= 0.5': [{'DELINQcat_1 <= 0.5': [{'CLAGE <= 0.1409180342963911': [True,
           False]},
         True]},
       {'CLAGE <= -0.004829010145782589': [True,
         {'MORTDUE <= -0.4517524860008637': [False, True]}]}]},
     {'CLAGE <= 0.2533275824463007': [{'MORTDUE <= 0.21021445335388034': [{'DEBTINC <= 1.5396192182451973': [False,
           True]},
         False]},
       {'MORTDUE <= -0.8982435570328543': [{'MORTDUE <= -0.9663206410117353': [False,
           True]},
         False]}]}]}]},
 {'CLAGE <= -0.30547252331620656': [{'NINQ <= 1.839290268

In [21]:
predictions = pd.DataFrame(random_forest_predictions(X, forest))

In [22]:
error = np.mean(np.array(predictions) == np.array(y))

error

0.87751677852349

The tree is able to classify a good proportion of the test set well even with only a depth of 3.

## Random Forest with Sci-Kit Learn

As mentioned, the package Ski-Kit Learn has prebuilt functions which avoid coding most algorithms by scratch. Now armed with the knowledge of the inner workings of the decision tree, it should be easier to understand what this algorithm is trying to achieve.

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=800, n_features=4,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X, y)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=2, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [24]:
pred_sklearn = clf.predict(X)

In [25]:
pred_sklearn[0:5]

array([0, 0, 1, 1, 0])

In [26]:
error_sklearn = (pred_sklearn == y).mean()


error_sklearn

0.9125

### SK Random Forest with Different Feature Space

If we allow for more features to be used, we may get better results but with all features. However, using too many features is essentially creating very similar trees which decreases the effectiveness of this ensemble method.

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=800, n_features=7,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)

clf = RandomForestClassifier(max_depth=5, random_state=0)
clf.fit(X, y)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [28]:
pred_sklearn2 = clf.predict(X)

In [29]:
error_sklearn2 = (pred_sklearn2 == y).mean()

error_sklearn2

0.92125

Questions to think about: if we are using bagging and random feature spaces correctly, how are we avoiding the common machine learning problem of overfitting?